In [27]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import sproc, udf
from snowflake.snowpark.dataframe_reader import *
from snowflake.snowpark.functions import *
from snowflake.snowpark.window import *
from snowflake.snowpark.types import StringType, StructType, StructField
import pandas as pd

In [28]:
connection_parameters = {
    "account": "",
    "user": "",
    "password": "",
    "warehouse": "", # optional
    "database":"COMMONS",
    "schema":"UTILS"
}  
session = Session.builder.configs(connection_parameters).create()  

### Create a stage to store your python script

In [29]:
query = "create or replace stage commons.utils.external_py_scripts" +\
        " directory = (enable = true)" +\
        " copy_options = (on_error='skip_file')"
        
session.sql(query).collect()


[Row(status='Stage area EXTERNAL_PY_SCRIPTS successfully created.')]

#### Import your local python script into stage

In [30]:
put_result = session.file.put("./string_utils.py", "@external_py_scripts/",auto_compress=False, overwrite=True)
put_result[0].status

'UPLOADED'

#### Add your stages to your session

This step is needed to get access to the stages and it's files from your current session which will be used for creating and executing python procedures and udfs

In [59]:
session.add_import("@external_py_scripts/string_utils.py")

#### Create python stored procedure to utilize the external script and return the count of vowels

In [32]:

@sproc(name="test_py_imports", is_permanent=False)
def test_py_imports(session: Session, strval: str) -> str:

    from string_utils import vowel_count

    return vowel_count(strval)

#### Test the python stored procedure

In [33]:
test_py_imports("this is a test sentence in english")

'10'

Next let us test the same custom script from a Python UDF

In [58]:
# let's use this table to test udf
df = session.table('SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER').select('C_COMMENT').limit(5)
df.show()

------------------------------------------------------
|"C_COMMENT"                                         |
------------------------------------------------------
|l theodolites boost slyly at the platelets: per...  |
| beans. fluffily regular packages                   |
|fully pending deposits sleep quickly. blithely ...  |
| furiously above the ironic packages. slyly bra...  |
|express instructions sleep quickly. ironic brai...  |
------------------------------------------------------



#### Create python stored procedure to utilize the external script and return the count of vowels

In [52]:
@udf(name="get_vowel_count", is_permanent=False, replace=True)
def get_vowel_count(strval: str) -> str:
    from string_utils import vowel_count
    return vowel_count(strval)

In [53]:
df = session.sql("select C_COMMENT, get_vowel_count(C_COMMENT) as VOWEL_COUNT from SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER limit 10")           

In [54]:
df.show()

----------------------------------------------------------------------
|"C_COMMENT"                                         |"VOWEL_COUNT"  |
----------------------------------------------------------------------
|l theodolites boost slyly at the platelets: per...  |24             |
| beans. fluffily regular packages                   |10             |
|fully pending deposits sleep quickly. blithely ...  |32             |
| furiously above the ironic packages. slyly bra...  |36             |
|express instructions sleep quickly. ironic brai...  |24             |
| carefully quickly even theodolites. boldly         |13             |
|bold packages. regular sheaves mold. blit           |12             |
|nal courts. carefully regular Tiresias lose qui...  |29             |
|efully even dependencies haggle furiously along...  |30             |
|fter the quickly silent requests. slyly special...  |29             |
----------------------------------------------------------------------



In [60]:
session.close()